In [1]:
import pandas as pd 

In [2]:
ds = pd.read_csv("antwerp_ds_modified.csv", index_col=0) 

In [3]:
ds

,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,BS 64,BS 65,BS 66,BS 67,BS 68,RX Time,SF,HDOP,Latitude,Longitude
0,-200,-200,-200,-200,-200,-200,-200,-200,-114,-200,...,-200,-200,-200,-200,-200,2017-12-28 16:14:07.247000+00:00,12,1.02,51.216736,4.412130
1,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2017-11-28 12:01:02.380000+00:00,7,0.57,51.232037,4.405185
2,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2017-12-14 09:05:22.832000+00:00,9,1.00,51.202763,4.390477
3,-200,-100,-200,-200,-200,-200,-200,-200,-200,-113,...,-200,-200,-200,-200,-200,2018-01-17 09:11:49.222000+00:00,12,0.62,51.194584,4.417887
4,-200,-106,-107,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2018-01-10 14:47:01.379000+00:00,12,1.00,51.207203,4.403470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123523,-200,-103,-106,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2018-01-04 10:57:15.856000+00:00,12,0.61,51.201839,4.414310
123524,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2018-01-03 23:17:34.799000+00:00,8,1.10,51.233677,4.416892
123525,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2017-12-15 10:13:56.383000+00:00,12,0.82,51.215370,4.412519
123526,-200,-97,-200,-200,-200,-200,-200,-200,-200,-92,...,-200,-200,-200,-200,-200,2018-01-12 23:33:09.229000+00:00,12,0.61,51.194252,4.418354


In [4]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [5]:
def get_weather_info(lat, long, time:str):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": long,
        "elevation": 5.0,
        "start_hour": time,
        "end_hour": time,
        "hourly": ["temperature_2m", 
                "relative_humidity_2m",
                "dew_point_2m", 
                "apparent_temperature", 
                "precipitation", 
                "rain", 
                "snowfall", 
                "snow_depth", 
                "weather_code", 
                "pressure_msl", 
                "surface_pressure", 
                "cloud_cover", 
                "cloud_cover_low", 
                "cloud_cover_mid", 
                "cloud_cover_high", 
                "et0_fao_evapotranspiration", 
                "vapour_pressure_deficit", 
                "wind_speed_10m", 
                "wind_speed_100m", 
                "wind_direction_10m", 
                "wind_direction_100m", 
                "wind_gusts_10m", 
                "shortwave_radiation", 
                "direct_radiation",
                "diffuse_radiation", 
                "direct_normal_irradiance", 
                "global_tilted_irradiance", 
                "terrestrial_radiation", 
                "shortwave_radiation_instant",
                "direct_radiation_instant", 
                "diffuse_radiation_instant", 
                "direct_normal_irradiance_instant", 
                "global_tilted_irradiance_instant", 
                "terrestrial_radiation_instant"]
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
    hourly_rain = hourly.Variables(5).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
    hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
    hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
    hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
    hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
    hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
    hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
    hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
    hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
    hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
    hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
    hourly_shortwave_radiation = hourly.Variables(22).ValuesAsNumpy()
    hourly_direct_radiation = hourly.Variables(23).ValuesAsNumpy()
    hourly_diffuse_radiation = hourly.Variables(24).ValuesAsNumpy()
    hourly_direct_normal_irradiance = hourly.Variables(25).ValuesAsNumpy()
    hourly_global_tilted_irradiance = hourly.Variables(26).ValuesAsNumpy()
    hourly_terrestrial_radiation = hourly.Variables(27).ValuesAsNumpy()
    hourly_shortwave_radiation_instant = hourly.Variables(28).ValuesAsNumpy()
    hourly_direct_radiation_instant = hourly.Variables(29).ValuesAsNumpy()
    hourly_diffuse_radiation_instant = hourly.Variables(30).ValuesAsNumpy()
    hourly_direct_normal_irradiance_instant = hourly.Variables(31).ValuesAsNumpy()
    hourly_global_tilted_irradiance_instant = hourly.Variables(32).ValuesAsNumpy()
    hourly_terrestrial_radiation_instant = hourly.Variables(33).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["rain"] = hourly_rain
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["snow_depth"] = hourly_snow_depth
    hourly_data["weather_code"] = hourly_weather_code
    hourly_data["pressure_msl"] = hourly_pressure_msl
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["cloud_cover"] = hourly_cloud_cover
    hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
    hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
    hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
    hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
    hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
    hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
    hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
    hourly_data["direct_radiation"] = hourly_direct_radiation
    hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
    hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
    hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
    hourly_data["terrestrial_radiation"] = hourly_terrestrial_radiation
    hourly_data["shortwave_radiation_instant"] = hourly_shortwave_radiation_instant
    hourly_data["direct_radiation_instant"] = hourly_direct_radiation_instant
    hourly_data["diffuse_radiation_instant"] = hourly_diffuse_radiation_instant
    hourly_data["direct_normal_irradiance_instant"] = hourly_direct_normal_irradiance_instant
    hourly_data["global_tilted_irradiance_instant"] = hourly_global_tilted_irradiance_instant
    hourly_data["terrestrial_radiation_instant"] = hourly_terrestrial_radiation_instant

    hourly_dataframe = pd.DataFrame(data = hourly_data)
    return hourly_dataframe

In [6]:
get_weather_info(ds["Latitude"][0], ds["Longitude"][0], "2017-12-28T16:14")

Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
0,2017-12-28 16:14:00+00:00,3.2455,74.766975,-0.8045,-1.471424,0.0,0.0,0.0,0.0,1.0,...,5.0,51.51376,7.999999,21.081083,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Convert the RX Time values to datetime format ISO8601 and store in a separate column upto minutes precision
# Expected format "2017-12-28T16:14"
ds["Weather param sense time"] = pd.to_datetime(ds["RX Time"], format='ISO8601').dt.strftime('%Y-%m-%dT%H:%M')

In [8]:
f"{ds["Weather param sense time"][0]}"

'2017-12-28T16:14'

In [9]:
def get_weather_info_mod(ds_row):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": ds_row["Latitude"],
        "longitude": ds_row["Longitude"],
        "elevation": 5.0,
        "start_hour": f"{ds_row["Weather param sense time"]}",
        "end_hour": f"{ds_row["Weather param sense time"]}",
        "hourly": ["temperature_2m", 
                "relative_humidity_2m",
                "dew_point_2m", 
                "apparent_temperature", 
                "precipitation", 
                "rain", 
                "snowfall", 
                "snow_depth", 
                "weather_code", 
                "pressure_msl", 
                "surface_pressure", 
                "cloud_cover", 
                "cloud_cover_low", 
                "cloud_cover_mid", 
                "cloud_cover_high", 
                "et0_fao_evapotranspiration", 
                "vapour_pressure_deficit", 
                "wind_speed_10m", 
                "wind_speed_100m", 
                "wind_direction_10m", 
                "wind_direction_100m", 
                "wind_gusts_10m", 
                "shortwave_radiation", 
                "direct_radiation",
                "diffuse_radiation", 
                "direct_normal_irradiance", 
                "global_tilted_irradiance", 
                "terrestrial_radiation", 
                "shortwave_radiation_instant",
                "direct_radiation_instant", 
                "diffuse_radiation_instant", 
                "direct_normal_irradiance_instant", 
                "global_tilted_irradiance_instant", 
                "terrestrial_radiation_instant"]
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
    hourly_rain = hourly.Variables(5).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
    hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
    hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
    hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
    hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
    hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
    hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
    hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
    hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
    hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
    hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
    hourly_shortwave_radiation = hourly.Variables(22).ValuesAsNumpy()
    hourly_direct_radiation = hourly.Variables(23).ValuesAsNumpy()
    hourly_diffuse_radiation = hourly.Variables(24).ValuesAsNumpy()
    hourly_direct_normal_irradiance = hourly.Variables(25).ValuesAsNumpy()
    hourly_global_tilted_irradiance = hourly.Variables(26).ValuesAsNumpy()
    hourly_terrestrial_radiation = hourly.Variables(27).ValuesAsNumpy()
    hourly_shortwave_radiation_instant = hourly.Variables(28).ValuesAsNumpy()
    hourly_direct_radiation_instant = hourly.Variables(29).ValuesAsNumpy()
    hourly_diffuse_radiation_instant = hourly.Variables(30).ValuesAsNumpy()
    hourly_direct_normal_irradiance_instant = hourly.Variables(31).ValuesAsNumpy()
    hourly_global_tilted_irradiance_instant = hourly.Variables(32).ValuesAsNumpy()
    hourly_terrestrial_radiation_instant = hourly.Variables(33).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["rain"] = hourly_rain
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["snow_depth"] = hourly_snow_depth
    hourly_data["weather_code"] = hourly_weather_code
    hourly_data["pressure_msl"] = hourly_pressure_msl
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["cloud_cover"] = hourly_cloud_cover
    hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
    hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
    hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
    hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
    hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
    hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
    hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
    hourly_data["direct_radiation"] = hourly_direct_radiation
    hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
    hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
    hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
    hourly_data["terrestrial_radiation"] = hourly_terrestrial_radiation
    hourly_data["shortwave_radiation_instant"] = hourly_shortwave_radiation_instant
    hourly_data["direct_radiation_instant"] = hourly_direct_radiation_instant
    hourly_data["diffuse_radiation_instant"] = hourly_diffuse_radiation_instant
    hourly_data["direct_normal_irradiance_instant"] = hourly_direct_normal_irradiance_instant
    hourly_data["global_tilted_irradiance_instant"] = hourly_global_tilted_irradiance_instant
    hourly_data["terrestrial_radiation_instant"] = hourly_terrestrial_radiation_instant

    hourly_dataframe = pd.DataFrame(data = hourly_data, index=[ds_row.name])  # Here we set index to the value of the original row index
    # this will help us concatinating the weather data to the main dataset
    # the index are only identifier of each sample and their corresponding weather data 
    return hourly_dataframe

In [10]:
ds

,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,BS 65,BS 66,BS 67,BS 68,RX Time,SF,HDOP,Latitude,Longitude,Weather param sense time
0,-200,-200,-200,-200,-200,-200,-200,-200,-114,-200,...,-200,-200,-200,-200,2017-12-28 16:14:07.247000+00:00,12,1.02,51.216736,4.412130,2017-12-28T16:14
1,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2017-11-28 12:01:02.380000+00:00,7,0.57,51.232037,4.405185,2017-11-28T12:01
2,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2017-12-14 09:05:22.832000+00:00,9,1.00,51.202763,4.390477,2017-12-14T09:05
3,-200,-100,-200,-200,-200,-200,-200,-200,-200,-113,...,-200,-200,-200,-200,2018-01-17 09:11:49.222000+00:00,12,0.62,51.194584,4.417887,2018-01-17T09:11
4,-200,-106,-107,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2018-01-10 14:47:01.379000+00:00,12,1.00,51.207203,4.403470,2018-01-10T14:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123523,-200,-103,-106,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2018-01-04 10:57:15.856000+00:00,12,0.61,51.201839,4.414310,2018-01-04T10:57
123524,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2018-01-03 23:17:34.799000+00:00,8,1.10,51.233677,4.416892,2018-01-03T23:17
123525,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2017-12-15 10:13:56.383000+00:00,12,0.82,51.215370,4.412519,2017-12-15T10:13
123526,-200,-97,-200,-200,-200,-200,-200,-200,-200,-92,...,-200,-200,-200,-200,2018-01-12 23:33:09.229000+00:00,12,0.61,51.194252,4.418354,2018-01-12T23:33


In [11]:
# Sample of n are tested for getting weather data
sample_ds = ds.sample(n=4000)
rdf = pd.concat([get_weather_info_mod(sample_ds.iloc[i, :]) for i in  range(len(sample_ds))], axis=0)

Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


OpenMeteoRequestsError: {'error': True, 'reason': 'Hourly API request limit exceeded. Please try again in the next hour.'}

In [12]:
import time
import datetime


# Sample of n are tested for getting weather data
sample_ds = ds.iloc[:4000]

rdf = pd.DataFrame()

i = 0
while i < len(sample_ds):

    try:
        rdf = pd.concat([rdf, get_weather_info_mod(sample_ds.iloc[i, :])], axis=0)
        print(f'saving data{i}')
        rdf.to_csv('sample_weather_data.csv')
        i += 1  
    except Exception as e:
        if 'Daily API request limit exceeded. Please try again tomorrow' in str(e):
            print(f'waiting untill {datetime.datetime.now() + datetime.timedelta(days=1)}')
            time.sleep(datetime.timedelta(days=1).total_seconds())

        elif 'Hourly API request limit exceeded. Please try again in the next hour' in str(e):
            print(f'waiting untill {datetime.datetime.now() + datetime.timedelta(hours=1)}')
            time.sleep(datetime.timedelta(hours=1).total_seconds())

        elif 'Minutely API request limit exceeded. Please try again in one minute' in str(e):
            print(f'waiting untill {datetime.datetime.now() + datetime.timedelta(minutes=1)}')
            time.sleep(datetime.timedelta(minutes=1).total_seconds())

        else:
            raise e

Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
saving data0
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
saving data1
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
saving data2
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
saving data3
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
saving data4
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
saving data5
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
saving data6
Coordinates 51.212650299072266°N 4

In [13]:
rdf

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
0,2017-12-28 16:14:00+00:00,3.245500,74.766975,-0.8045,-1.471424,0.0,0.0,0.00,0.00,1.0,...,5.0,51.513760,7.999999,21.081083,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2017-11-28 12:01:00+00:00,6.695500,85.262260,4.3955,3.050040,0.0,0.0,0.00,0.00,1.0,...,79.0,356.052460,184.999985,418.422516,182.836151,104.760170,78.075974,356.052490,182.836136,413.528412
2,2017-12-14 09:05:00+00:00,4.045500,79.654655,0.8455,-2.085968,0.0,0.0,0.00,0.01,2.0,...,29.0,54.315105,35.000004,133.442535,54.010590,9.258959,44.751633,63.470764,46.219566,205.923157
3,2018-01-17 09:11:00+00:00,3.245500,72.081696,-1.3045,-3.012658,0.2,0.0,0.14,0.00,71.0,...,24.0,170.033157,44.999992,164.430283,66.079582,30.837139,35.242443,180.502975,62.246716,241.455215
4,2018-01-10 14:47:00+00:00,7.445500,94.019669,6.5455,4.709943,0.1,0.1,0.00,0.00,51.0,...,35.0,11.516722,37.000000,245.607712,26.762775,1.446637,25.316139,11.516721,26.762775,177.652542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,2018-01-09 11:16:00+00:00,5.395500,83.626175,2.8455,1.746084,0.0,0.0,0.00,0.00,1.0,...,69.0,519.156372,206.000000,373.244537,219.702377,146.112747,73.589630,519.156311,219.702377,398.071472
3996,2018-02-04 02:48:00+00:00,0.995500,94.049133,0.1455,-3.728485,0.0,0.0,0.00,0.00,3.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3997,2018-01-05 11:38:00+00:00,8.695499,80.883629,5.5955,4.968346,0.1,0.1,0.00,0.00,51.0,...,96.0,29.777212,104.000000,380.047272,107.800163,8.292320,99.507843,29.777210,107.800163,393.934204
3998,2018-01-23 08:19:00+00:00,6.345500,91.710983,5.0955,2.636001,0.0,0.0,0.00,0.00,3.0,...,1.0,0.000000,1.000000,54.419956,2.323495,0.000000,2.323495,0.000000,1.103674,126.444511


In [14]:
sample_ds


,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,BS 65,BS 66,BS 67,BS 68,RX Time,SF,HDOP,Latitude,Longitude,Weather param sense time
0,-200,-200,-200,-200,-200,-200,-200,-200,-114,-200,...,-200,-200,-200,-200,2017-12-28 16:14:07.247000+00:00,12,1.02,51.216736,4.412130,2017-12-28T16:14
1,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2017-11-28 12:01:02.380000+00:00,7,0.57,51.232037,4.405185,2017-11-28T12:01
2,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2017-12-14 09:05:22.832000+00:00,9,1.00,51.202763,4.390477,2017-12-14T09:05
3,-200,-100,-200,-200,-200,-200,-200,-200,-200,-113,...,-200,-200,-200,-200,2018-01-17 09:11:49.222000+00:00,12,0.62,51.194584,4.417887,2018-01-17T09:11
4,-200,-106,-107,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2018-01-10 14:47:01.379000+00:00,12,1.00,51.207203,4.403470,2018-01-10T14:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-200,-200,-200,-200,-200,-200,-200,-200,-114,-200,...,-200,-200,-200,-200,2018-01-09 11:16:00.472000+00:00,12,0.62,51.233234,4.428070,2018-01-09T11:16
3996,-200,-200,-200,-200,-95,-200,-200,-200,-103,-200,...,-200,-200,-200,-200,2018-02-04 02:48:22.505000+00:00,10,0.88,51.235592,4.417192,2018-02-04T02:48
3997,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2018-01-05 11:38:00.888000+00:00,8,0.57,51.209614,4.432734,2018-01-05T11:38
3998,-200,-200,-200,-200,-200,-200,-107,-200,-111,-200,...,-200,-200,-200,-200,2018-01-23 08:19:10.118000+00:00,12,0.57,51.227322,4.439460,2018-01-23T08:19


In [15]:
ds_weather = pd.concat([sample_ds, rdf], axis=1, join='outer')

In [16]:
ds_weather

,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
0,-200,-200,-200,-200,-200,-200,-200,-200,-114,-200,...,5.0,51.513760,7.999999,21.081083,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,79.0,356.052460,184.999985,418.422516,182.836151,104.760170,78.075974,356.052490,182.836136,413.528412
2,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,29.0,54.315105,35.000004,133.442535,54.010590,9.258959,44.751633,63.470764,46.219566,205.923157
3,-200,-100,-200,-200,-200,-200,-200,-200,-200,-113,...,24.0,170.033157,44.999992,164.430283,66.079582,30.837139,35.242443,180.502975,62.246716,241.455215
4,-200,-106,-107,-200,-200,-200,-200,-200,-200,-200,...,35.0,11.516722,37.000000,245.607712,26.762775,1.446637,25.316139,11.516721,26.762775,177.652542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-200,-200,-200,-200,-200,-200,-200,-200,-114,-200,...,69.0,519.156372,206.000000,373.244537,219.702377,146.112747,73.589630,519.156311,219.702377,398.071472
3996,-200,-200,-200,-200,-95,-200,-200,-200,-103,-200,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3997,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,96.0,29.777212,104.000000,380.047272,107.800163,8.292320,99.507843,29.777210,107.800163,393.934204
3998,-200,-200,-200,-200,-200,-200,-107,-200,-111,-200,...,1.0,0.000000,1.000000,54.419956,2.323495,0.000000,2.323495,0.000000,1.103674,126.444511


In [17]:
ds_weather.to_csv('sample_antwerp_ds_weather-data.csv')